In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Will use only the first and the second GPU devices

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.io
from mpl_toolkits.mplot3d import Axes3D

from networks.InfoSSM import *

In [ ]:
B=50
K=4
T=20
M=20
Dx=4
Dy=2
Din=2
Dout=2
Nc = 3 # 1 for PR-SSM
dt=0.1
lr=1e-3

In [ ]:
Data = scipy.io.loadmat('./Dubins.mat')
yseq = Data["Ytrain"].reshape((-1,1,T,Dy))
yseq = yseq + 0.01*np.random.normal(0,1,yseq.shape)
Ntrain_all = yseq.shape[0]
print(yseq.shape)

In [ ]:
SSMNet = InfoSSM(K, T, M, Dx, Dy, Nc, dt, lr)

In [ ]:
info_lambda = B*T # 0.0 for H-GPSSM
loss_hist = []
bound_hist = []
info_hist = []

In [ ]:
epoch_begin = len(loss_hist)
for epoch in range(epoch_begin,30001):    
    batch_idx = np.random.choice(Ntrain_all,B)
    yseq_batch = yseq[batch_idx,:,:,:]
    
    losses = SSMNet.train(yseq_batch, info_lambda)
    loss_hist.append(losses[0])
    bound_hist.append(losses[0]+info_lambda*losses[2])
    info_hist.append(losses[2])
    
    if epoch % 50 == 0:
        print (epoch, losses[0], losses[0]+info_lambda*losses[2], losses[2])

    if epoch % 500 == 0:
        yseq_samples = SSMNet.sess.run(SSMNet.yseq_samples, feed_dict={SSMNet.yseq_true:yseq_batch, SSMNet.info_lambda:info_lambda}) # (B,K,T,Dy)
        
        for b in range(0,B,10):
            fig = plt.figure()
            for k in range(K):
                plt.plot(yseq_samples[b,k,:,0], yseq_samples[b,k,:,1])
                plt.plot(yseq_samples[b,k,0,0], yseq_samples[b,k,0,1], '.')
            plt.plot(yseq_batch[b,0,:,0], yseq_batch[b,0,:,1],'k-')
            plt.plot(yseq_batch[b,0,0,0], yseq_batch[b,0,0,1],'k.')
            plt.axis('equal')
            plt.show()
        
        xseq, code = SSMNet.sess.run([SSMNet.xseq_samples, SSMNet.c0_samples], feed_dict={SSMNet.yseq_true:yseq}) # (B,K,T,Dy)
        plt.figure()
        for nc in range(Nc):
            plt.plot(code[:,0,0,nc])

        for b in range(0,Ntrain_all,7):
            plt.figure()
            px = yseq[b,0,:,0]
            py = yseq[b,0,:,1]
            plt.plot(px, py)
            plt.plot(px[0], py[0],'.')
            plt.axis('equal')
            plt.show()
            print(code[b,0,0,:])